In [31]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

In [32]:
# 구글드라이브 연결
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [33]:
# 데이터파일 불러오기
df = pd.read_csv('/gdrive/My Drive/3-2_machine_learning/image-10k.csv', header=None)

X = df.iloc[:, 1:].values     # 데이터샘플
y = df.iloc[:, 0].values      # 타겟레이블

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=1, stratify=y)

print(x_train.shape, y_train.shape)

(5000, 784) (5000,)


In [34]:
import cv2

hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (7, 7), 9)

hog_x_train = np.zeros((5000,324),dtype="float64")
hog_x_test = np.zeros_like(hog_x_train)

for i in range(5000):
  hog_x_train[i]=hog.compute(x_train[i].reshape(28,28).astype(np.uint8)).squeeze()

for i in range(5000):
  hog_x_test[i]=hog.compute(x_test[i].reshape(28,28).astype(np.uint8)).squeeze()

In [37]:
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(weights='distance',algorithm='kd_tree',p=1,n_neighbors=3)

from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(100,10), activation='relu', batch_size=500,
                    solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-1)

from sklearn import svm
svm_clf = svm.SVC(kernel="rbf",C=10,probability=True)

vo_clf = VotingClassifier(estimators=[('knn', knn),('mlp', mlp),('svm',svm_clf)],  voting='soft')

# Voting분류기 학습
svm_clf.fit(hog_x_train, y_train)
pred_y = svm_clf.predict(hog_x_test)
print('corse 분류기 정확도', accuracy_score(y_test, pred_y))

corse 분류기 정확도 0.8722
